In [27]:
import tqdm
import json
import requests
import numpy as np
from lxml import etree
from argparse import ArgumentParser
import sys
from fuzzywuzzy import fuzz

In [23]:
def get_top_n_different(text, candidates, n=3):
    if len(candidates) == n:
        print(text, candidates)
        return candidates
    elif len(candidates) < n:
        return candidates + [None for i in range(n - len(candidates))]

    top_n = list()
    
    for c in candidates:
        top_n.append(fuzz.ratio(text, c))
        
    return np.array(top_n).argsort()[-n:][::-1]

In [24]:
dbpedia_json = None

with open('../datasets/DBpedia/smarttask_dbpedia_train.json') as json_file:
    dbpedia_json = json.load(json_file)

In [25]:
dbpedia_json[0]

{'id': 'dbpedia_1177',
 'question': 'Was Jacqueline Kennedy Onassis a follower of Melkite Greek Catholic Church?',
 'category': 'boolean',
 'type': ['boolean']}

In [41]:
augmented_json = list()

for q in tqdm.tqdm(dbpedia_json):
    qstr = q['question']

    r = requests.get('http://www.google.com/complete/search', params={'q':qstr, 'hl':'ja', 'ie':'utf_8', 'oe':'utf_8', 'output': 'toolbar'})
    n = 3
    
    try:
        root = etree.XML(r.text)
        sugs = root.xpath('//suggestion')
        sugstrs = [s.get('data') for s in sugs]
        candidates = get_top_n_different(qstr, sugstrs, n)
    except:
        candidates = [None for i in range(n)]
    finally:
        root = etree.XML(r.text)
        sugs = root.xpath('//suggestion')
        sugstrs = [s.get('data') for s in sugs]
        candidates = get_top_n_different(qstr, sugstrs, n)
        
    for i in range(n):
        q['question_{0}'.format(i+1)] = candidates[i]
        
    augmented_json.append(q)

  0%|          | 0/17571 [00:00<?, ?it/s]


XMLSyntaxError: AttValue: " or ' expected, line 1, column 253 (<string>, line 1)

In [30]:
with open('../datasets/DBpedia/suggestions_dbpedia.json', 'w') as json_file:
    json.dump(augmented_json, json_file)

TypeError: Object of type 'int64' is not JSON serializable

In [40]:
augmented_json[1]

{'id': 'dbpedia_14427',
 'question': 'What is the name of the opera based on Twelfth Night ?',
 'category': 'resource',
 'type': ['dbo:Opera', 'dbo:MusicalWork', 'dbo:Work'],
 'question_1': None,
 'question_2': None,
 'question_3': None}

In [43]:
import requests, json 
URL="http://suggestqueries.google.com/complete/search?client=firefox&q=Hel Wor" 
headers = {'User-agent':'Mozilla/5.0'} 
response = requests.get(URL, headers=headers) 

In [44]:
response.text

'<html><head><meta http-equiv="content-type" content="text/html; charset=utf-8"/><title>Sorry...</title><style> body { font-family: verdana, arial, sans-serif; background-color: #fff; color: #000; }</style></head><body><div><table><tr><td><b><font face=sans-serif size=10><font color=#4285f4>G</font><font color=#ea4335>o</font><font color=#fbbc05>o</font><font color=#4285f4>g</font><font color=#34a853>l</font><font color=#ea4335>e</font></font></b></td><td style="text-align: left; vertical-align: bottom; padding-bottom: 15px; width: 50%"><div style="border-bottom: 1px solid #dfdfdf;">Sorry...</div></td></tr></table></div><div style="margin-left: 4em;"><h1>We\'re sorry...</h1><p>... but your computer or network may be sending automated queries. To protect our users, we can\'t process your request right now.</p></div><div style="margin-left: 4em;">See <a href="https://support.google.com/websearch/answer/86640">Google Help</a> for more information.<br/><br/></div><div style="text-align: ce

In [46]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']


"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '../../Загрузки/AutoML-466fc88403db.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))


ValueError: Client secrets must be for a web or installed app.